October 21st 2020  
Run some basic symmetric pruning using the new framework

In [1]:
import sys, os, glob 
import datetime, time

import numpy as np
from numpy import linalg as nla
import matplotlib.pyplot as plt

gen_fn_dir = os.path.abspath('..') + '/general_fns/'
sys.path.append(gen_fn_dir)

from general_file_fns import load_pickle_file, save_pickle_file
import linear_net_fns as lnf
import lin_alg_fns as laf
import undirected_graph_fns as gf
import combined_sparsification_fns as csf

%matplotlib inline

curr_date=datetime.datetime.now().strftime('%Y_%m_%d')+'_'

sd = int((time.time()%1)*(2**31))
np.random.seed(sd)

In [2]:
# Make connectivity matrix
t0 = time.time()
n_nodes = 2000
matrix_type = 'clustered'

#these parameters are only relevant for the matrix_type = 'clustered' case
cl_1 = 1000 #size of first cluster
cl_2 = 200 
cl_3 = 800
sr_density = 0.6 #density of connections within clusters
lr_tot = 5000 #total number of connections across clusters

A_non_dd = gf.make_imb_clustered_graph([cl_1,cl_2,cl_3], sr_density, lr_tot)

A = laf.shift_to_make_diagonally_dominant(A_non_dd, shift_type='individual', diag_type='neg')

ew_orig, ev_orig = laf.sort_eig(A, ret_ev=True, hermitian=True)

print('Time to make matrix ', time.time()-t0)
print(np.min(ew_orig), np.max(ew_orig))

Time to make matrix  10.852299213409424
-1.2273827477650583 -0.03182701350810497


In [3]:
# Get the normalized probs
desired_edge_frac = 0.20
n_edges_orig = (np.count_nonzero(A) - n_nodes) 

desired_mean_num_edges = desired_edge_frac * n_edges_orig # Leave as a float for now since used in normalization
print(desired_mean_num_edges)
prob_normalization = {'norm_type' : 'sum', 'norm_val' : desired_mean_num_edges, 'max_val' : 1}
prob_matrix, inv_diff_matrix = csf.get_sparsify_probs(A, normalization=prob_normalization, return_diffs=True,
                                                             matrix_type='symmetric')
prob_matrix_up = np.triu(prob_matrix)

202120.0


In [4]:
# Now to sparsify
# Set seed
sd = int((time.time()%1)*(2**31))
np.random.seed(sd)

A_sparse_np = csf.sparsify_given_probs(A, prob_matrix_up, symmetric=True, diagonal_type='row_sum')
n_edges_sparse = (np.count_nonzero(A_sparse_np) - n_nodes)
print('Number of edges original {}, new {}, ratio {}, desired density {}'.format(
    n_edges_orig, n_edges_sparse, n_edges_sparse/n_edges_orig, desired_edge_frac))

Maximum rescale  815.9982015616018
Number of edges original 1010600, new 197404, ratio 0.19533346526815754, desired density 0.2


In [5]:
print(nla.norm(A_sparse_np - A_sparse_np.T))

0.0


In [6]:
# Control
desired_mean_num_edges = desired_edge_frac * n_edges_orig # Leave as a float for now since used in normalization
print(desired_mean_num_edges)
prob_normalization = {'norm_type' : 'sum', 'norm_val' : desired_mean_num_edges, 'max_val' : 1}
wt_prob_matrix = csf.get_sparsify_probs_control(A, normalization=prob_normalization)
wt_prob_matrix_up = np.triu(wt_prob_matrix)


202120.0


In [7]:
# Control pruning
# Set seed
sd = int((time.time()%1)*(2**31))
np.random.seed(sd)

A_sparse_wbc = csf.sparsify_given_probs(A, wt_prob_matrix_up, symmetric=True, diagonal_type='row_sum')
n_edges_sparse = (np.count_nonzero(A_sparse_wbc) - n_nodes)
print('Number of edges original {}, new {}, ratio {}, desired density {}'.format(
    n_edges_orig, n_edges_sparse, n_edges_sparse/n_edges_orig, desired_edge_frac))

Maximum rescale  703.9745037034415
Number of edges original 1010600, new 201664, ratio 0.19954878290124678, desired density 0.2


In [8]:
# Calculate and compare the spectra of pruned to unpruned networks
spect_np = csf.compare_spectrum(A_sparse_np, ew_orig, ev_orig, ret_ew=True)
spect_wbc = csf.compare_spectrum(A_sparse_wbc, ew_orig, ev_orig, ret_ew=True)

In [ ]:
# Now simulate the three networks (original, noise-prune, control) with these inputs. 
ev_as_input = False #False corresponds to random input
dt = 5e-3
n_steps = int(8e3+1) #number of timesteps, n_steps*dt = total time simulated for
sim_times = dt * np.arange(n_steps)
sig = 0.01 #standard deviation of Gaussian white noise used in simulation
sim_params = {'dt' : dt, 'n_steps' : n_steps, 'noise_std' : sig} #, 'input' : input_b, 'ics' : ics}

n_sims = 20

list_of_rel_diff_wbc_mag_across_time = []
list_of_rel_diff_np_mag_across_time = []

t0 = time.time()
for curr_sim in range(n_sims):
    # Randomize for initial conditions (only relevant if using random ICs)
    sd = int((time.time()%1)*(2**31))
    np.random.seed(sd)
    
    # Set initial conditions and background input
    if ev_as_input:
        # for a small index i, ev_orig[:-i] has eigenvalue closer to 0 [i.e., slow time-scales]
        sim_params['ics'] = ev_orig[:, -curr_sim-1]
        # Make bg input the same as eigenvector to keep on eigenvector direction
        sim_params['input'] = np.tile(sim_params['ics'], (n_steps, 1))
    else:
        sim_params['ics'] = np.random.rand(n_nodes)
        # sim_params['input'] = np.tile(sim_params['ics'], (n_steps, 1))
        sim_params['input'] = 0.0002 * np.ones((n_steps, n_nodes))
    
    sd = int((time.time()%1)*(2**31))
    
    # Reset the seed before each so that the noise is always the same
    np.random.seed(sd)
    x_orig = lnf.sim_linear_net(A, sim_params)
    
    np.random.seed(sd)
    x_np = lnf.sim_linear_net(A_sparse_np, sim_params)

    np.random.seed(sd)
    x_wbc = lnf.sim_linear_net(A_sparse_wbc, sim_params)

    #compute ||x_{orig} - x_{wbc}||_2 and ||x_{orig} - x_{np}||_2
    diff_wbc_matrix = x_orig - x_wbc
    diff_np_matrix = x_orig - x_np
    diff_wbc_mag_across_time = nla.norm(diff_wbc_matrix, axis=1)
    diff_np_mag_across_time = nla.norm(diff_np_matrix, axis=1)

    #compute ||x_{orig}||_2 and ||x_{orig}||_2 in order to compute relative errors
    sum_wbc_mag_across_time = nla.norm(x_orig, axis=1) 
    sum_np_mag_across_time = nla.norm(x_orig, axis=1)

    list_of_rel_diff_wbc_mag_across_time.append(diff_wbc_mag_across_time/sum_wbc_mag_across_time)
    list_of_rel_diff_np_mag_across_time.append(diff_np_mag_across_time/sum_np_mag_across_time)

    print('Sim {}, time {}'.format(curr_sim, time.time()-t0))

mean_rel_diff_wbc_mag_across_time = np.mean(list_of_rel_diff_wbc_mag_across_time, axis=0)
mean_rel_diff_np_mag_across_time = np.mean(list_of_rel_diff_np_mag_across_time, axis=0)
std_rel_diff_wbc_mag_across_time = np.std(list_of_rel_diff_wbc_mag_across_time, axis=0)
std_rel_diff_np_mag_across_time = np.std(list_of_rel_diff_np_mag_across_time, axis=0)

Sim 0, time 55.232897996902466
Sim 1, time 112.98052835464478
Sim 2, time 169.53374123573303
Sim 3, time 230.79451894760132
Sim 4, time 289.82171297073364
Sim 5, time 347.5210871696472


In [ ]:
# Save spectrum data and dynamics data
data = {'n_nodes': n_nodes, 'matrix_type': matrix_type,
    'spect_np_eps_ew' : np.abs(spect_np['eps_ew']),'spect_wbc_eps_ew' : np.abs(spect_wbc['eps_ew']), 
    'spect_np_eps_ev' : np.abs(spect_np['eps_ev']),'spect_wbc_eps_ev' : np.abs(spect_wbc['eps_ev']),
    'spect_np_S_ev_angle' : np.abs(spect_np['S_ev_angle']),'spect_wbc_S_ev_angle' : np.abs(spect_wbc['S_ev_angle']),
    'mean_rel_diff_np_mag_across_time' : mean_rel_diff_np_mag_across_time, 'mean_rel_diff_wbc_mag_across_time' : mean_rel_diff_wbc_mag_across_time,
    'std_rel_diff_np_mag_across_time' : std_rel_diff_np_mag_across_time, 'std_rel_diff_wbc_mag_across_time' : std_rel_diff_wbc_mag_across_time}

# save_pickle_file(data, 'n_{}_{}_{}_ev_input_{}_data.p'.format(n_nodes, 'clustered', 'symm', ev_as_input))

graph_name = 'n_{}_{}_{}_ev_input_{}'.format(n_nodes, 'clustered', 'symm', ev_as_input)

os.makedirs('{}'.format(graph_name)) 

In [ ]:
# Plot the spectra
fig, ax = plt.subplots(1,3,figsize=(16,6))
ax[0].boxplot([data['spect_np_eps_ew'], data['spect_wbc_eps_ew']], labels = ['noise-prune', 'weights']);
ax[0].set_ylabel('$\epsilon_{\lambda_i}$', fontsize = 14);
ax[1].boxplot([data['spect_np_eps_ev'], data['spect_wbc_eps_ev']], labels = ['noise-prune', 'weights']);
ax[1].set_ylabel('$\epsilon_{v_i}$', fontsize = 14);
ax[2].boxplot([data['spect_np_S_ev_angle'], data['spect_wbc_S_ev_angle']], labels = ['noise-prune', 'weights']);
ax[2].set_ylabel('$\cos{\Theta_i}$', fontsize = 14);

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(13,6))
for i, x in enumerate(list_of_rel_diff_np_mag_across_time):
    ax.plot(sim_times, x, color='r')
for i, x in enumerate(list_of_rel_diff_wbc_mag_across_time):
    ax.plot(sim_times, x, color='b')

In [ ]:
fig, ax = plt.subplots( figsize=(3,3)) #should be 3,3#
ax.plot(sim_times, data['mean_rel_diff_wbc_mag_across_time'], label='weight-based-control', color='#1034A6')
ax.fill_between(sim_times, data['mean_rel_diff_wbc_mag_across_time']- data['std_rel_diff_wbc_mag_across_time'], data['mean_rel_diff_wbc_mag_across_time'] + data['std_rel_diff_wbc_mag_across_time'], color='#1034A6', alpha=0.3)
ax.plot(sim_times, data['mean_rel_diff_np_mag_across_time'], label='noise-prune', color='#A91101')
ax.fill_between(sim_times, data['mean_rel_diff_np_mag_across_time'] - data['std_rel_diff_np_mag_across_time'], data['mean_rel_diff_np_mag_across_time'] + data['std_rel_diff_np_mag_across_time'] , color='#A91101', alpha=0.3)
ax.set_xticks([0, 10, 20, 30, 40])
ax.set_yticks([0, 1.2])


fig.savefig('{}/{}_rel_error2norm.pdf'.format(graph_name,graph_name), transparent=False,  bbox_inches='tight', dpi=1000)
#ff.ref_axes(ax, labels=False)
#fig.savefig('{}/{}_rel_error2norm.pdf'.format(graph_name,graph_name), transparent=False,  bbox_inches='tight', dpi=1000)
